In [3]:
from myosuite.utils import gym

MyoSuite:> Registering Myo Envs


In [4]:
import mujoco
env = gym.make('myoHandPoseRandom-v0', normalize_act = False)

    MyoSuite: A contact-rich simulation suite for musculoskeletal motor control
        Vittorio Caggiano, Huawei Wang, Guillaume Durandau, Massimo Sartori, Vikash Kumar
        L4DC-2019 | https://sites.google.com/view/myosuite
    


In [5]:
import numpy as np
env.env.init_qpos[:] = np.zeros(len(env.env.init_qpos),)
mjcModel = env.env.sim.model

In [6]:

print("Actuators:")
for i in range(mjcModel.nu):
    print(f"[{i}, {mjcModel.id2name(i, mujoco.mjtObj.mjOBJ_ACTUATOR)}]", end=",\t\t")
    if (i+1) % 5 == 0 and i != 0:
        print()

print("\nJoints:")
for i in range(mjcModel.njnt):
    print(f"[{i}, {mjcModel.id2name(i, mujoco.mjtObj.mjOBJ_JOINT)}]", end=",\t\t")
    if (i+1) % 5 == 0 and i != 0:
        print()

Actuators:
[0, ECRL],		[1, ECRB],		[2, ECU],		[3, FCR],		[4, FCU],		
[5, PL],		[6, PT],		[7, PQ],		[8, FDS5],		[9, FDS4],		
[10, FDS3],		[11, FDS2],		[12, FDP5],		[13, FDP4],		[14, FDP3],		
[15, FDP2],		[16, EDC5],		[17, EDC4],		[18, EDC3],		[19, EDC2],		
[20, EDM],		[21, EIP],		[22, EPL],		[23, EPB],		[24, FPL],		
[25, APL],		[26, OP],		[27, RI2],		[28, LU_RB2],		[29, UI_UB2],		
[30, RI3],		[31, LU_RB3],		[32, UI_UB3],		[33, RI4],		[34, LU_RB4],		
[35, UI_UB4],		[36, RI5],		[37, LU_RB5],		[38, UI_UB5],		
Joints:
[0, pro_sup],		[1, deviation],		[2, flexion],		[3, cmc_abduction],		[4, cmc_flexion],		
[5, mp_flexion],		[6, ip_flexion],		[7, mcp2_flexion],		[8, mcp2_abduction],		[9, pm2_flexion],		
[10, md2_flexion],		[11, mcp3_flexion],		[12, mcp3_abduction],		[13, pm3_flexion],		[14, md3_flexion],		
[15, mcp4_flexion],		[16, mcp4_abduction],		[17, pm4_flexion],		[18, md4_flexion],		[19, mcp5_flexion],		
[20, mcp5_abduction],		[21, pm5_flexion],		[22, md5_flexion],		

In [7]:
mjcModel.actuator('FDP2').id

15

In [8]:
mjcModel.opt.timestep

0.002

In [9]:
from IPython.display import HTML
from base64 import b64encode
 
def show_video(video_path, video_width = 400):
   
  video_file = open(video_path, "r+b").read()
 
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video autoplay width={video_width} controls><source src="{video_url}"></video>""")


In [11]:
# 오프스크린 프레임버퍼 크기 설정
mjcModel.vis.global_.offwidth = 1920
mjcModel.vis.global_.offheight = 1080

In [37]:
key_frame_interval = 29.97
video_fps = 30
# steps_per_frame = int(1/(mjcModel.opt.timestep * video_fps))
steps_per_frame = 1 / (mjcModel.opt.timestep * video_fps) # 0.002(1/500), 30fps이면 1초 500step인데 이것을 30으로 나누면 16.666step에 1frame
# 1 / (sec/step * frame / sec) = step / frame
playback_duration = 10
total_step_count = round(playback_duration / mjcModel.opt.timestep)
frames = []

env.reset()
next_frame_step = 0
for step_idx in range(total_step_count):
    # if not(step_idx%steps_per_frame):
    # if (step_idx * video_fps * mjcModel.opt.timestep) % 1 < mjcModel.opt.timestep:
    if step_idx >= next_frame_step:
        frame = env.sim.renderer.render_offscreen(
                                    width=1920,
                                    height=1080,
                                    camera_id=3)
        frames.append(frame)
        next_frame_step += steps_per_frame

    ctrl = np.zeros(mjcModel.na,)
    # ctrl[mjcModel.actuator('FDP2').id] = 0.2
    ctrl[mjcModel.actuator('EDC2').id] = 0.2

    env.step(ctrl)
print(f"total_step_count: {total_step_count}, len(frames): {len(frames)}")
import os
import skvideo.io

os.makedirs('temp_videos', exist_ok=True)
# make a local copy
output_dict = {
    "-pix_fmt": "yuv420p",
    "-r": f"{video_fps}",
    "-g": f"{key_frame_interval}",
    "-video_track_timescale": "30000",
    "-b:v": "1M",
    "-bufsize": "2M" ,
    # "-map": "0",
    # "-vcodec": "libx264",
    # "-f": "mpegts"
}
skvideo.io.vwrite('temp_videos/myosuite_study_01.mp4', np.asarray(frames),outputdict=output_dict)

# show in the notebook
show_video('temp_videos/myosuite_study_01.mp4', video_width=1920)

total_step_count: 5000, len(frames): 300


In [23]:
f"{video_fps}"

'30'